## Structure of the Swatch Class

The Swatch class serves as the Model in the MVVM pattern and is used by the PaletteViewModel to manage individual color data in a .NET MAUI app.

INotifyPropertyChanged: https://learn.microsoft.com/en-us/dotnet/api/system.componentmodel.inotifypropertychanged?view=net-9.0

This class implements the interface that notifies the UI a property value has changed. In this regards the Swatch Class must have a way to raise an event when a property has changed such as button values and swatch values when the user interacts with the application

OnPropertyChanged() for example

In [ ]:
public class Swatch : INotifyPropertyChanged

The following are private fields used to store the state of the swatch:

In [ ]:
private Color _color;
private bool _isFavorite;
private bool _isLocked;
// private bool _isHovered; -> To use for future hover implementation
private bool _isDeleted;
private Color _previousColor;

The color property have a get and set methods:
+ get => _color means return what is sotre din the private _color field when asked
+ set contains the logic when the user tries to change the Color property (in this context, presses the regenerate button)
    + It checks first to proceed if the swatch is not locked
    + It also checks if the color is different from the current
        + When both conditions are true, the private field is updated with a new color and then OnPropertyChanged() notifies the UI so it can update visually on the swatch

In [ ]:
public Color Color
{
    get => _color;
    set
    {
        if (!_isLocked && _color != value)
        {
            _color = value;
            OnPropertyChanged();
        }
    }
}

Similarly, PreviousColor is implemented to hold the value when a delete action is executed

In [ ]:
public Color PreviousColor
{
    get => _previousColor;
    set
    {
        _previousColor = value;
        OnPropertyChanged();
    }
}

The following properties return specific icons (as ImageSource) based on the state of related boolean flags. Each property is read-only and does not include a setter; the value is derived from the corresponding boolean property.

In [ ]:
// One of the Image Property
public string HexCode => Color.ToHex(); //returns hexcode for color
public ImageSource FavoriteColor => ImageSource.FromFile(IsFavoriteColor ? "heart.png" : "unheart.png");
public ImageSource LockImage => ImageSource.FromFile(IsLocked ? "padlock.png" : "unlock.png");
public ImageSource DeleteImage => ImageSource.FromFile(IsDeleted ? "undo.png" : "delete.png");

Boolean Button Property
<break>

Controls button visibility based on the state of the swatch.
- ButtonVisible: Returns true only when the swatch is not deleted and is active.
- DeleteButtonVisible: Returns true if the swatch is either active or deleted.
<break>

Behavior:
When a user clicks on a swatch, buttons are conditionally displayed.
- If the swatch has been deleted or deselected (i.e., inactive), ButtonVisible will return false, hiding the associated buttons.
- DeleteButtonVisible ensures the delete option remains available as long as the swatch is either active or has already been deleted.

In [ ]:
public bool ButtonVisible => !IsDeleted && IsActive;
public bool DeleteButtonVisible => IsActive || IsDeleted;

The following boolean properties (connected to the image property above) represent the state of a swatch and drive the UI's dynamic behavior. When a state-changing action occurs (e.g., favoriting a swatch), the corresponding image property updates automatically to reflect the new state (e.g., switching to a filled heart icon).

In [ ]:
// One of the Boolean Property
public bool IsFavoriteColor
{
    get => _isFavoriteColor;
    set
    {
        if (_isFavoriteColor != value)
        {
            _isFavoriteColor = value;
            OnPropertyChanged();
            OnPropertyChanged(nameof(FavoriteColor));
        }
    }
}

[ICommand](https://learn.microsoft.com/en-us/dotnet/api/system.windows.input.icommand?view=net-9.0)s connect UI actions (buttons) to code method. 


In [ ]:
public ICommand ToggleFavoriteCommand { get; }
public ICommand ToggleLockCommand { get; }
public ICommand ToggleDeleteCommand { get; }
public ICommand ToggleActivateCommand { get; }

The Swatch Construcotr initializes the swatch with initial colors and the commands to execute when user interacts with the UI

In [ ]:
public Swatch(Color color)
{
    _color = color;

    ToggleFavoriteCommand = new Command(() => IsFavorite = !IsFavorite);
    ToggleLockCommand = new Command(() => IsLocked = !IsLocked);
    ToggleDeleteCommand = new Command(ToggleDelete);
    ToggleActivateCommand = new Command(() => IsActive = !IsActive);
}

The following method have logic for a Delete command with an Undo option. It will switch buttons states when interacted as well as toggle the buttons visibility when the swatch is clicked.

In [ ]:
// One of the methods
private void ToggleDelete()
{
    if (IsDeleted)
    {
        Color = _previousColor;  
        IsDeleted = false;
        IsActive = false;
        //OnPropertyChanged(nameof(Color));
    }
    else
    {
        _previousColor = Color;  
        Color = Colors.Transparent; 
        IsDeleted = true;
        IsActive = false;
        //OnPropertyChanged(nameof(Color));
    }

    OnPropertyChanged(nameof(Color));
    OnPropertyChanged(nameof(ButtonVisible));
    OnPropertyChanged(nameof(DeleteButtonVisible));
}

Interface: [INotifyPropertyChange](https://learn.microsoft.com/en-us/dotnet/api/system.runtime.compilerservices.callermembernameattribute?view=net-9.0) implementation using OnPropertyChanged method with CallerMemberName attribute:

"Implementing the INotifyPropertyChanged interface when binding data. This interface allows the property of an object to notify a bound control that the property has changed, so that the control can display the updated information."

[CallerMemberName] automatically fills the parameter with the name of the method OnPropertyChanged.

In [ ]:
public event PropertyChangedEventHandler PropertyChanged;

protected void OnPropertyChanged([CallerMemberName] string name = null) =>
    PropertyChanged?.Invoke(this, new PropertyChangedEventArgs(name));